In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_mean = pd.read_csv('../data/tomato_mean.csv')
raw_stderr = pd.read_csv('../data/tomato_stderr.csv')

In [3]:
# drop_cols = ['CULTIVAR','SEASON','HARVEST WEEK','HARVEST DATE','MEASURE', *raw_stderr.columns[raw_stderr.isna().any()]]

drop_cols = ['Variety','panel number','panel date','measure', *raw_stderr.columns[raw_stderr.isna().any()]]
df_mean = raw_mean.drop(columns=drop_cols)
df_mean = df_mean.replace('-', np.nan).astype(float)
df_mean = df_mean.fillna(df_mean.mean())

df_stderr = raw_stderr.drop(columns=drop_cols)
df_stderr = df_stderr.replace('-', np.nan).astype(float)
df_stderr = df_stderr.fillna(df_stderr.mean())

drop_cols

['Variety', 'panel number', 'panel date', 'measure', 'Bitter']

In [4]:
def normalize_column(col):
    return (col - col.mean()) / col.std()

In [5]:
# NUM_LABEL_COLS = 5
NUM_LABEL_COLS = 7

In [6]:
val_dataset = df_mean.copy()
val_dataset[val_dataset.columns[:NUM_LABEL_COLS]] = val_dataset[val_dataset.columns[:NUM_LABEL_COLS]] / 100
val_dataset[val_dataset.columns[NUM_LABEL_COLS:]] = val_dataset[val_dataset.columns[NUM_LABEL_COLS:]].apply(normalize_column, axis=0)

val_dataset
# val_dataset.to_csv('../data/strawberry_val_dataset.csv')
val_dataset.to_csv('../data/tomato_val_dataset.csv')

In [7]:
val_dataset

,Overall Liking,Texture liking,Sweetness,Sourness,Salty,Umami,Tomato Flavor Intenstity,glucose,fructose,Soluble solids,...,citric:malic,3-methyl-1-pentanol,2-ethylfuran,isopentyl acetate,cis-3-hexenyl acetate,benzothiazole,benzyl alcohol,citric acid,3-methyl-2-butenal,p-anisaldehyde
0,0.3367,0.3406,0.2551,0.1820,0.1294,0.1286,0.3341,2.280922,1.913190,2.562857,...,2.510053,1.266061,-0.092380,-1.007032,1.312008,1.039829,0.453284,0.923551,-0.017934,-0.340967
1,0.2878,0.2747,0.2556,0.1653,0.1256,0.1667,0.3561,0.738719,0.686613,0.719634,...,-0.582452,1.458910,-0.868766,2.501611,0.512250,1.471306,-0.547379,-1.264632,0.926590,1.559112
2,0.2701,0.2099,0.2009,0.1710,0.1111,0.1352,0.3089,0.027918,-0.164139,0.099564,...,-0.663188,0.968021,1.149837,0.293926,-0.335493,-0.055458,-0.070873,-0.613602,0.526984,-0.229198
3,0.2649,0.2119,0.2505,0.1673,0.1153,0.0896,0.3026,2.796207,3.405277,2.928103,...,-0.247226,-0.609840,-0.247657,1.713152,-0.303503,-0.088648,-0.428252,1.945306,0.853935,-0.452737
4,0.2575,0.2325,0.2440,0.1290,0.1040,0.1019,0.3024,1.215633,1.071788,1.246269,...,-0.508738,-0.013759,-0.713489,-0.928186,-1.119256,0.309638,-0.213824,-0.215751,0.236361,-0.452737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.0532,0.1300,0.0685,0.0689,0.0647,0.0877,0.1470,-1.159797,-0.771818,-1.004671,...,-1.108986,-0.925412,1.305114,-0.533956,-0.943309,0.475591,-0.404427,-1.978956,-0.381213,1.670882
75,0.0473,0.0923,0.0831,0.0711,0.0615,0.0812,0.1462,-0.682884,-0.420299,-0.945212,...,-0.961556,-0.557244,-1.179320,-0.218573,0.688197,-0.088648,-0.785631,-1.165169,-0.272229,-0.564506
76,0.0453,0.0643,0.1329,0.1183,0.0762,0.0908,0.2257,0.601675,0.232255,1.823869,...,3.649120,-0.820221,-1.645152,-0.888763,-1.631101,-0.254601,2.478433,1.556496,1.471508,0.106110
77,0.0434,0.0631,0.0649,0.2046,0.1077,0.0881,0.2468,-1.695183,-1.502903,-1.004671,...,1.388537,1.669292,-0.713489,4.196797,-0.143551,-0.652887,0.238856,2.768135,0.236361,-0.340967


In [8]:
df_samples = pd.DataFrame()

# For each row in the original DataFrames
for idx in range(len(df_mean)):
    # Get mean and standard error for the current row
    mean_row = df_mean.iloc[idx]
    stderr_row = df_stderr.iloc[idx]

    # Generate 10 samples from the normal distribution with the mean and standard error
    samples = pd.DataFrame({column: np.random.normal(mean, np.divide(stderr, mean, out=np.zeros_like(mean), where=mean!=0), 1000) for column, mean, stderr in zip(df_mean.columns, mean_row, stderr_row)})

    # Append the samples to the new DataFrame
    df_samples = pd.concat([df_samples, samples], ignore_index=True)

df_samples

,Overall Liking,Texture liking,Sweetness,Sourness,Salty,Umami,Tomato Flavor Intenstity,glucose,fructose,Soluble solids,...,citric:malic,3-methyl-1-pentanol,2-ethylfuran,isopentyl acetate,cis-3-hexenyl acetate,benzothiazole,benzyl alcohol,citric acid,3-methyl-2-butenal,p-anisaldehyde
0,33.891391,34.017116,25.692837,18.135491,13.139556,13.026196,33.505584,26.809714,27.820207,8.117250,...,22.734081,1.337285,0.144830,0.000000,2.351023,0.343823,0.989488,4.757869,0.293036,-0.028634
1,33.651416,34.005523,25.610373,17.998463,13.188897,12.616284,33.314516,26.812271,27.874345,8.104075,...,22.775576,1.288996,0.038219,0.000000,2.403691,0.315755,0.421329,4.695491,0.313327,0.070866
2,33.658351,33.961930,25.415212,18.064141,12.766382,12.914376,33.395599,26.837679,27.762818,8.086822,...,22.769707,1.392661,0.069378,0.000000,2.390543,-0.115973,-0.504827,4.719552,0.331193,0.150675
3,33.392021,33.958823,25.660841,18.208522,12.708957,12.875156,33.346352,26.792229,27.826923,8.063018,...,22.700789,1.389522,0.073256,0.000000,2.366902,0.235368,-0.011646,4.723975,0.272752,-0.052732
4,33.762744,33.924369,25.499137,18.237000,12.952523,12.990090,33.518869,26.845420,27.801500,8.059712,...,22.694543,1.314111,0.066563,0.000000,2.298290,0.195168,0.454180,4.642748,0.374545,-0.188303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78995,3.723830,-1.264498,13.542418,8.972571,8.416288,9.113873,19.141202,7.519759,8.654572,4.916899,...,2.589233,0.947147,0.219645,0.491705,0.042724,0.123221,0.873095,2.665852,0.207973,-0.047231
78996,3.106944,1.516883,13.565505,8.862883,8.103012,9.613919,18.927532,7.422428,8.678328,4.892621,...,2.583019,0.836388,0.214474,1.066181,1.043465,0.079773,0.358801,2.635448,0.300187,0.170650
78997,4.665842,4.316196,13.381052,9.075774,8.297704,9.821542,19.327019,7.382006,8.704994,4.846322,...,2.483816,0.195597,0.385258,-0.803672,1.276154,0.160335,0.283534,2.586498,0.468832,0.077246
78998,4.313525,0.588031,13.344874,8.942868,8.395796,9.917917,19.219694,7.375598,8.623330,4.936663,...,2.527848,1.359413,0.468958,-0.630754,0.761080,0.073387,0.552719,2.569956,0.259590,0.096498


In [11]:
sample_dataset = df_samples.copy()
sample_dataset[sample_dataset.columns[:NUM_LABEL_COLS]] = sample_dataset[sample_dataset.columns[:NUM_LABEL_COLS]].clip(0) / 100
sample_dataset[sample_dataset.columns[NUM_LABEL_COLS:]] = sample_dataset[sample_dataset.columns[NUM_LABEL_COLS:]].apply(normalize_column, axis=0)

# sample_dataset.to_csv('../data/strawberry_samples_big.csv')
sample_dataset.to_csv('../data/tomato_samples_big.csv')
sample_dataset

,Overall Liking,Texture liking,Sweetness,Sourness,Salty,Umami,Tomato Flavor Intenstity,glucose,fructose,Soluble solids,...,citric:malic,3-methyl-1-pentanol,2-ethylfuran,isopentyl acetate,cis-3-hexenyl acetate,benzothiazole,benzyl alcohol,citric acid,3-methyl-2-butenal,p-anisaldehyde
0,0.338914,0.340171,0.256928,0.181355,0.131396,0.130262,0.335056,2.289596,1.929062,2.592386,...,2.520910,1.093684,0.136266,-0.571344,1.210977,1.037262,1.072747,0.980180,-0.146406,-0.119881
1,0.336514,0.340055,0.256104,0.179985,0.131889,0.126163,0.333145,2.290067,1.939249,2.581130,...,2.528238,1.017167,-0.366412,-0.571344,1.290722,0.932396,0.002128,0.923562,-0.089412,0.212553
2,0.336584,0.339619,0.254152,0.180641,0.127664,0.129144,0.333956,2.294738,1.918264,2.566390,...,2.527202,1.181431,-0.219492,-0.571344,1.270815,-0.680615,-1.743087,0.945401,-0.039227,0.479200
3,0.333920,0.339588,0.256608,0.182085,0.127090,0.128752,0.333464,2.286382,1.930326,2.546053,...,2.515032,1.176457,-0.201208,-0.571344,1.235020,0.632055,-0.813754,0.949416,-0.203380,-0.200393
4,0.337627,0.339244,0.254991,0.182370,0.129525,0.129901,0.335189,2.296162,1.925542,2.543229,...,2.513928,1.056964,-0.232769,-0.571344,1.131131,0.481864,0.064032,0.875690,0.082542,-0.653343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78995,0.037238,0.000000,0.135424,0.089726,0.084163,0.091139,0.191412,-1.257119,-1.677233,-0.141795,...,-1.036429,0.475485,0.489022,0.524832,-2.284104,0.213054,0.853422,-0.918645,-0.385336,-0.182016
78996,0.031069,0.015169,0.135655,0.088629,0.081030,0.096139,0.189275,-1.275015,-1.672763,-0.162537,...,-1.037527,0.299981,0.464641,1.805531,-0.768846,0.050725,-0.115696,-0.946242,-0.126320,0.545937
78997,0.046658,0.043162,0.133811,0.090758,0.082977,0.098215,0.193270,-1.282447,-1.667745,-0.202092,...,-1.055045,-0.715392,1.269892,-2.362999,-0.416522,0.351719,-0.257527,-0.990672,0.347381,0.233870
78998,0.043135,0.005880,0.133449,0.089429,0.083958,0.099179,0.192197,-1.283625,-1.683112,-0.124910,...,-1.047269,1.128747,1.664543,-1.977505,-1.196415,0.026865,0.249716,-1.005686,-0.240351,0.298191
